# **Algorimto Disjktra**

In [ ]:
import heapq

def dijkstra(graph, start):

  estado = {}
  for i in graph:
    estado[i] = (float('inf'),None)
  estado[start] = (0,None)

  hq = []
  heapq.heappush(hq,(0,start))

  while hq:
    u = heapq.heappop(hq)
    for v in graph[u[1]]:
      oldist = estado[v[0]][0]
      newdist = estado[u[1]][0] + v[1]
      if newdist < oldist:
        estado[v[0]] = (newdist, u[1])
        if oldist == float('inf'):
          heapq.heappush(hq,(0,v[0]))
        else:
          heapq.heappush(hq,(newdist,v[0]))
  return estado

def printDist(estado):
    """
    Imprime el estado de Dijkstra con formato legible.
    estado: {nodo: (distancia, padre, visitado)}
    """
    print("\n" + "="*40)
    print(f"{'Nodo':<10} {'Distancia':<15} {'Padre':<10} ")
    print("="*40)

    for nodo, (distancia, padre) in estado.items():
        # Formatear distancia
        dist_str = str(distancia) if distancia != float('inf') else "∞"

        # Formatear padre
        padre_str = str(padre) if padre is not None else "-"

        print(f"{str(nodo):<10} {dist_str:<15} {padre_str:<10}")

    print("="*40 + "\n")


# Versión simple (una línea por nodo)
def printDistSimple(estado):
    """Versión simple de impresión"""
    print("\nResultado de Dijkstra:")
    for nodo, (distancia, padre) in estado.items():
        dist = distancia if distancia != float('inf') else "∞"
        print(f"  Nodo {nodo}: distancia={dist}, padre={padre}")


# Versión con camino completo
def printCamino(estado, nodo_destino):
    """Imprime el camino desde el inicio hasta un nodo destino"""
    if estado[nodo_destino][0] == float('inf'):
        print(f"No hay camino hasta {nodo_destino}")
        return

    camino = []
    actual = nodo_destino

    while actual is not None:
        camino.append(actual)
        actual = estado[actual][1]  # padre

    camino.reverse()

    print(f"Camino: {' → '.join(map(str, camino))}")
    print(f"Distancia total: {estado[nodo_destino][0]}")



def main():

  grafo = {
    'S': [('B', 1), ('C', 6)],
    'B': [('T', 5), ('E', 1)],
    'C': [('E', 1)],
    'T': [],
    'E': [('T', 1)]
  }

  tabla = dijkstra(graph=grafo,start="S")

  printDistSimple(tabla)

  printDist(tabla)

  grafo = {
      'S': [('2', 9), ('6', 14), ('7', 15)],
      '2': [('3', 23)],
      '3': [('t', 11)],
      '4': [('3', 6), ('t', 6)],
      '5': [('4', 11)],
      '6': [('5', 30), ('3', 18), ('7', 5)],
      '7': [('5', 20), ('t', 44)],
      't': [],
  }
  tabla = dijkstra(graph=grafo,start="S")

  printDistSimple(tabla)

  printDist(tabla)

main()


Resultado de Dijkstra:
  Nodo S: distancia=0, padre=None
  Nodo B: distancia=1, padre=S
  Nodo C: distancia=6, padre=S
  Nodo T: distancia=3, padre=E
  Nodo E: distancia=2, padre=B

Nodo       Distancia       Padre      
S          0               -         
B          1               S         
C          6               S         
T          3               E         
E          2               B         


Resultado de Dijkstra:
  Nodo S: distancia=0, padre=None
  Nodo 2: distancia=9, padre=S
  Nodo 3: distancia=32, padre=2
  Nodo 4: distancia=46, padre=5
  Nodo 5: distancia=35, padre=7
  Nodo 6: distancia=14, padre=S
  Nodo 7: distancia=15, padre=S
  Nodo t: distancia=43, padre=3

Nodo       Distancia       Padre      
S          0               -         
2          9               S         
3          32              2         
4          46              5         
5          35              7         
6          14              S         
7          15              S         


# **Disjoint Set Structure**

In [ ]:
"""
Disjoint Set (Union-Find) con optimizaciones:
- Union by Rank
- Path Compression

Complejidad amortizada: O(α(n)) ≈ O(1) para todas las operaciones
donde α es la función inversa de Ackermann
"""

class Info:
    """
    Clase auxiliar que almacena información de cada elemento
    """
    def __init__(self, elem):
        if elem is None:
            raise ValueError("Element cannot be None")
        self.root = elem  # Padre del elemento (o sí mismo si es raíz)
        self.rank = 1     # Tamaño del subárbol

    def __repr__(self):
        return f"Info(root={self.root}, rank={self.rank})"


class DisjointSet:
    """
    Estructura de datos Disjoint Set (Union-Find) optimizada
    """

    def __init__(self, initial_set=None):
        """
        Inicializa el Disjoint Set

        Args:
            initial_set: Lista, set o iterable con elementos iniciales
        """
        self.parents_map = {}

        if initial_set is not None:
            for elem in initial_set:
                if elem is None:
                    raise ValueError("Elements cannot be None")
                if elem in self.parents_map:
                    raise ValueError(f"Duplicate element: {elem}")
                self.parents_map[elem] = Info(elem)

    def __len__(self):
        """Retorna el número de elementos en la estructura"""
        return len(self.parents_map)

    def __contains__(self, elem):
        """Verifica si un elemento existe en la estructura"""
        return elem in self.parents_map

    def __repr__(self):
        """Representación en string para debugging"""
        sets = self.get_all_sets()
        result = f"DisjointSet with {len(self)} elements in {len(sets)} sets:\n"
        for i, (root, members) in enumerate(sets.items(), 1):
            result += f"  Set {i} (root: {root}): {members}\n"
        return result

    @property
    def size(self):
        """Retorna el número de elementos"""
        return len(self.parents_map)

    def has(self, elem):
        """
        Verifica si un elemento existe

        Args:
            elem: Elemento a verificar

        Returns:
            bool: True si existe, False en caso contrario
        """
        return elem in self.parents_map

    def add(self, elem):
        """
        Agrega un nuevo elemento a la estructura

        Args:
            elem: Elemento a agregar

        Returns:
            bool: True si se agregó, False si ya existía

        Raises:
            ValueError: Si el elemento es None
        """
        if elem is None:
            raise ValueError("Element cannot be None")

        if elem in self.parents_map:
            return False

        self.parents_map[elem] = Info(elem)
        return True

    def find_partition(self, elem):
        """
        Encuentra la raíz del conjunto al que pertenece el elemento
        Aplica Path Compression para optimizar futuras búsquedas

        Args:
            elem: Elemento a buscar

        Returns:
            Raíz del conjunto

        Raises:
            KeyError: Si el elemento no existe
        """
        if elem not in self.parents_map:
            raise KeyError(f"Element not found: {elem}")

        info = self.parents_map[elem]

        # Si el elemento es su propia raíz, lo retornamos
        if info.root == elem:
            return elem

        # Path Compression: actualizar el padre para que apunte directamente a la raíz
        info.root = self.find_partition(info.root)
        return info.root

    def merge(self, elem1, elem2):
        """
        Une los conjuntos que contienen a elem1 y elem2
        Aplica Union by Rank para mantener árboles balanceados

        Args:
            elem1: Primer elemento
            elem2: Segundo elemento

        Returns:
            bool: True si se unieron, False si ya estaban en el mismo conjunto

        Raises:
            KeyError: Si algún elemento no existe
        """
        # Encontrar las raíces de ambos elementos
        root1 = self.find_partition(elem1)
        root2 = self.find_partition(elem2)

        # Si ya están en el mismo conjunto, no hacer nada
        if root1 == root2:
            return False

        info1 = self.parents_map[root1]
        info2 = self.parents_map[root2]

        # Union by Rank: el árbol más pequeño se cuelga del más grande
        if info1.rank >= info2.rank:
            # root1 es la nueva raíz
            info2.root = root1
            info1.rank += info2.rank
        else:
            # root2 es la nueva raíz
            info1.root = root2
            info2.rank += info1.rank

        return True

    def union(self, elem1, elem2):
        """Alias de merge para compatibilidad con terminología estándar"""
        return self.merge(elem1, elem2)

    def are_disjoint(self, elem1, elem2):
        """
        Verifica si dos elementos están en conjuntos diferentes

        Args:
            elem1: Primer elemento
            elem2: Segundo elemento

        Returns:
            bool: True si están en conjuntos diferentes
        """
        root1 = self.find_partition(elem1)
        root2 = self.find_partition(elem2)
        return root1 != root2

    def are_connected(self, elem1, elem2):
        """
        Verifica si dos elementos están en el mismo conjunto

        Args:
            elem1: Primer elemento
            elem2: Segundo elemento

        Returns:
            bool: True si están en el mismo conjunto
        """
        return not self.are_disjoint(elem1, elem2)

    def find(self, elem):
        """Alias de find_partition para compatibilidad"""
        return self.find_partition(elem)

    def get_set_members(self, elem):
        """
        Obtiene todos los elementos que pertenecen al mismo conjunto que elem

        NOTA: Esta operación es O(n) y rompe la eficiencia del disjoint set
        Úsala solo para debugging o cuando realmente necesites ver todos los elementos

        Args:
            elem: Elemento de referencia

        Returns:
            list: Lista con todos los elementos del conjunto
        """
        root = self.find_partition(elem)
        members = []

        for element in self.parents_map.keys():
            if self.find_partition(element) == root:
                members.append(element)

        return members

    def count_sets(self):
        """
        Retorna el número de conjuntos disjuntos

        NOTA: Esta operación es O(n)

        Returns:
            int: Número de conjuntos
        """
        roots = set()

        for elem in self.parents_map.keys():
            roots.add(self.find_partition(elem))

        return len(roots)

    def get_all_sets(self):
        """
        Retorna todos los conjuntos como un diccionario de raíz -> lista de elementos

        NOTA: Esta operación es O(n)

        Returns:
            dict: Diccionario con raíz como clave y lista de elementos como valor
        """
        sets = {}

        for elem in self.parents_map.keys():
            root = self.find_partition(elem)

            if root not in sets:
                sets[root] = []
            sets[root].append(elem)

        return sets

    def clear(self):
        """Limpia toda la estructura"""
        self.parents_map.clear()

    def get_roots(self):
        """
        Retorna todos los elementos que son raíces

        Returns:
            set: Conjunto de raíces
        """
        return {self.find_partition(elem) for elem in self.parents_map.keys()}


# ============================================================================
# TESTS UNITARIOS
# ============================================================================

class DisjointSetTests:
    """Suite completa de tests para DisjointSet"""

    def __init__(self):
        self.passed = 0
        self.failed = 0

    def assert_true(self, condition, test_name):
        """Verifica que una condición sea verdadera"""
        if condition:
            print(f"✓ {test_name}")
            self.passed += 1
        else:
            print(f"✗ {test_name}")
            self.failed += 1

    def assert_equal(self, actual, expected, test_name):
        """Verifica que dos valores sean iguales"""
        if actual == expected:
            print(f"✓ {test_name}")
            self.passed += 1
        else:
            print(f"✗ {test_name}: expected {expected}, got {actual}")
            self.failed += 1

    def test_basic_initialization(self):
        print("\n=== Test: Basic Initialization ===")

        ds = DisjointSet([1, 2, 3, 4, 5])
        self.assert_equal(len(ds), 5, "Size should be 5")
        self.assert_equal(ds.count_sets(), 5, "Should have 5 separate sets")
        self.assert_true(ds.has(1), "Should contain element 1")
        self.assert_true(ds.has(5), "Should contain element 5")
        self.assert_true(not ds.has(6), "Should not contain element 6")

    def test_add(self):
        print("\n=== Test: Add Operation ===")

        ds = DisjointSet()
        self.assert_true(ds.add("A"), "Should add new element A")
        self.assert_true(ds.add("B"), "Should add new element B")
        self.assert_true(not ds.add("A"), "Should not add duplicate A")
        self.assert_equal(len(ds), 2, "Size should be 2")

    def test_simple_merge(self):
        print("\n=== Test: Simple Merge ===")

        ds = DisjointSet([1, 2, 3])
        self.assert_true(ds.merge(1, 2), "Should merge 1 and 2")
        self.assert_true(ds.are_connected(1, 2), "1 and 2 should be connected")
        self.assert_true(ds.are_disjoint(1, 3), "1 and 3 should be disjoint")
        self.assert_equal(ds.count_sets(), 2, "Should have 2 sets")

    def test_chained_merge(self):
        print("\n=== Test: Chained Merge ===")

        ds = DisjointSet([1, 2, 3, 4, 5])
        ds.merge(1, 2)
        ds.merge(2, 3)
        ds.merge(3, 4)

        self.assert_true(ds.are_connected(1, 4), "1 and 4 should be connected")
        self.assert_true(ds.are_disjoint(1, 5), "1 and 5 should be disjoint")
        self.assert_equal(ds.count_sets(), 2, "Should have 2 sets")

    def test_merge_already_connected(self):
        print("\n=== Test: Merge Already Connected ===")

        ds = DisjointSet([1, 2, 3])
        self.assert_true(ds.merge(1, 2), "First merge should succeed")
        self.assert_true(not ds.merge(1, 2), "Second merge should return False")
        self.assert_true(not ds.merge(2, 1), "Reverse merge should also return False")

    def test_path_compression(self):
        print("\n=== Test: Path Compression ===")

        ds = DisjointSet([1, 2, 3, 4, 5])

        # Crear cadena larga: 1-2-3-4-5
        ds.merge(1, 2)
        ds.merge(2, 3)
        ds.merge(3, 4)
        ds.merge(4, 5)

        # Primera búsqueda de 5 (comprime el camino)
        root1 = ds.find_partition(5)

        # Segunda búsqueda debería ser más rápida (ya comprimido)
        root2 = ds.find_partition(5)

        self.assert_equal(root1, root2, "Roots should be the same")
        self.assert_true(ds.are_connected(1, 5), "1 and 5 should be connected")

    def test_union_by_rank(self):
        print("\n=== Test: Union by Rank ===")

        ds = DisjointSet([1, 2, 3, 4, 5, 6])

        # Crear dos árboles de diferente tamaño
        ds.merge(1, 2)
        ds.merge(1, 3)  # Árbol de 3 elementos

        ds.merge(4, 5)  # Árbol de 2 elementos

        # Unir los árboles
        ds.merge(1, 4)

        # Verificar que todos están conectados
        self.assert_true(ds.are_connected(1, 5), "1 and 5 should be connected")
        self.assert_true(ds.are_connected(2, 4), "2 and 4 should be connected")
        self.assert_true(ds.are_disjoint(1, 6), "1 and 6 should be disjoint")
        self.assert_equal(ds.count_sets(), 2, "Should have 2 sets")

    def test_with_strings(self):
        print("\n=== Test: String Elements ===")

        ds = DisjointSet(["Alice", "Bob", "Carol", "David"])

        ds.merge("Alice", "Bob")
        ds.merge("Carol", "David")

        self.assert_true(ds.are_connected("Alice", "Bob"),
                        "Alice and Bob should be connected")
        self.assert_true(ds.are_disjoint("Alice", "Carol"),
                        "Alice and Carol should be disjoint")

        ds.merge("Bob", "Carol")

        self.assert_true(ds.are_connected("Alice", "David"),
                        "Alice and David should be connected")
        self.assert_equal(ds.count_sets(), 1, "All should be in one set")

    def test_get_set_members(self):
        print("\n=== Test: Get Set Members ===")

        ds = DisjointSet([1, 2, 3, 4, 5])

        ds.merge(1, 2)
        ds.merge(2, 3)
        ds.merge(4, 5)

        set1 = ds.get_set_members(1)
        set2 = ds.get_set_members(4)

        self.assert_equal(len(set1), 3, "Set 1 should have 3 members")
        self.assert_equal(len(set2), 2, "Set 2 should have 2 members")
        self.assert_true(all(x in set1 for x in [1, 2, 3]),
                        "Set 1 should contain 1, 2, 3")
        self.assert_true(all(x in set2 for x in [4, 5]),
                        "Set 2 should contain 4, 5")

    def test_error_handling(self):
        print("\n=== Test: Error Handling ===")

        ds = DisjointSet([1, 2, 3])

        try:
            ds.find_partition(99)
            self.assert_true(False, "Should raise error for non-existent element")
        except KeyError:
            self.assert_true(True, "Should raise error for non-existent element")

        try:
            ds.merge(1, 99)
            self.assert_true(False, "Should raise error when merging non-existent")
        except KeyError:
            self.assert_true(True, "Should raise error when merging non-existent")

        try:
            DisjointSet([None])
            self.assert_true(False, "Should raise error for None element")
        except ValueError:
            self.assert_true(True, "Should raise error for None element")

    def test_large_scale(self):
        print("\n=== Test: Large Scale (Performance) ===")

        import time

        n = 10000
        elements = list(range(n))

        start = time.time()
        ds = DisjointSet(elements)
        end = time.time()
        print(f"  Initialized {n:,} elements in {(end - start) * 1000:.2f}ms")

        start = time.time()
        for i in range(n - 1):
            ds.merge(i, i + 1)
        end = time.time()
        print(f"  Merged {n - 1:,} pairs in {(end - start) * 1000:.2f}ms")

        import random
        start = time.time()
        for _ in range(1000):
            ds.find_partition(random.randint(0, n - 1))
        end = time.time()
        print(f"  1,000 random finds in {(end - start) * 1000:.2f}ms")

        self.assert_true(ds.are_connected(0, n - 1), "All elements should be connected")
        self.assert_equal(ds.count_sets(), 1, "Should have 1 set")

    def run_all_tests(self):
        print("\n╔════════════════════════════════════╗")
        print("║   DISJOINT SET TEST SUITE         ║")
        print("╚════════════════════════════════════╝")

        self.test_basic_initialization()
        self.test_add()
        self.test_simple_merge()
        self.test_chained_merge()
        self.test_merge_already_connected()
        self.test_path_compression()
        self.test_union_by_rank()
        self.test_with_strings()
        self.test_get_set_members()
        self.test_error_handling()
        self.test_large_scale()

        print("\n╔════════════════════════════════════╗")
        print(f"║   RESULTS                          ║")
        print(f"║   Passed: {str(self.passed).ljust(24)} ║")
        print(f"║   Failed: {str(self.failed).ljust(24)} ║")
        print("╚════════════════════════════════════╝\n")


# ============================================================================
# EJEMPLOS DE USO
# ============================================================================

def ejemplo_red_social():
    """Ejemplo 1: Red Social"""
    print("\n╔════════════════════════════════════╗")
    print("║   EJEMPLO 1: RED SOCIAL           ║")
    print("╚════════════════════════════════════╝\n")

    usuarios = ["Alice", "Bob", "Carol", "David", "Eve", "Frank"]
    red_social = DisjointSet(usuarios)

    print("Estado inicial:")
    print(red_social)

    # Crear amistades
    print("Creando amistades...")
    red_social.merge("Alice", "Bob")
    print("Alice y Bob son amigos")

    red_social.merge("Bob", "Carol")
    print("Bob y Carol son amigos")

    red_social.merge("David", "Eve")
    print("David y Eve son amigos")

    print("\nEstado después de amistades:")
    print(red_social)

    # Consultas
    print("Consultas:")
    print(f"¿Alice y Carol conectadas? {red_social.are_connected('Alice', 'Carol')}")
    print(f"¿Alice y David conectadas? {red_social.are_connected('Alice', 'David')}")
    print(f"¿Frank conectado con alguien? {red_social.are_connected('Frank', 'Alice')}")

    # Unir grupos
    print("\nUniendo grupos de Alice y David...")
    red_social.merge("Carol", "Eve")

    print("\nEstado final:")
    print(red_social)

    print(f"Amigos de Alice: {red_social.get_set_members('Alice')}")


def ejemplo_componentes_conectados():
    """Ejemplo 2: Componentes Conectados en Grafo"""
    print("\n╔════════════════════════════════════╗")
    print("║   EJEMPLO 2: GRAFO CONECTADO      ║")
    print("╚════════════════════════════════════╝\n")

    # Definir grafo como lista de aristas
    vertices = ["A", "B", "C", "D", "E", "F", "G", "H"]
    aristas = [
        ("A", "B"),
        ("B", "C"),
        ("C", "A"),  # Ciclo A-B-C
        ("D", "E"),
        ("F", "G"),
        ("G", "H")
    ]

    print(f"Vértices: {', '.join(vertices)}")
    print(f"Aristas: {', '.join([f'{u}-{v}' for u, v in aristas])}")

    # Encontrar componentes conectados
    grafo = DisjointSet(vertices)

    print("\nProcesando aristas...")
    for u, v in aristas:
        if grafo.merge(u, v):
            print(f"  Conectando {u} y {v}")
        else:
            print(f"  {u} y {v} ya estaban conectados (ciclo detectado)")

    print("\nComponentes conectados:")
    componentes = grafo.get_all_sets()
    for i, (root, members) in enumerate(componentes.items(), 1):
        print(f"  Componente {i}: {members}")

    print(f"\nTotal de componentes: {len(componentes)}")


def ejemplo_kruskal():
    """Ejemplo 3: Algoritmo de Kruskal (MST)"""
    print("\n╔════════════════════════════════════╗")
    print("║   EJEMPLO 3: KRUSKAL'S MST        ║")
    print("╚════════════════════════════════════╝\n")

    # Grafo ponderado: (peso, u, v)
    vertices = ["A", "B", "C", "D", "E"]
    aristas_ponderadas = [
        (1, "A", "B"),
        (4, "A", "C"),
        (2, "B", "C"),
        (3, "B", "D"),
        (5, "C", "E"),
        (6, "D", "E"),
        (7, "C", "D")
    ]

    print("Grafo:")
    for peso, u, v in aristas_ponderadas:
        print(f"  {u} --{peso}-- {v}")

    # Algoritmo de Kruskal
    mst = DisjointSet(vertices)
    aristas_seleccionadas = []
    peso_total = 0

    # Ordenar aristas por peso
    aristas_ordenadas = sorted(aristas_ponderadas, key=lambda x: x[0])

    print("\nProcesando aristas en orden de peso:")
    for peso, u, v in aristas_ordenadas:
        if mst.are_disjoint(u, v):
            mst.merge(u, v)
            aristas_seleccionadas.append((u, v, peso))
            peso_total += peso
            print(f"  ✓ Agregar {u}-{v} (peso: {peso})")
        else:
            print(f"  ✗ Saltar {u}-{v} (crearía ciclo)")

    print("\nÁrbol de Expansión Mínima:")
    for u, v, peso in aristas_seleccionadas:
        print(f"  {u} --{peso}-- {v}")
    print(f"\nPeso total del MST: {peso_total}")

ejemplo_red_social()
ejemplo_componentes_conectados()
ejemplo_kruskal()





╔════════════════════════════════════╗
║   EJEMPLO 1: RED SOCIAL           ║
╚════════════════════════════════════╝

Estado inicial:
DisjointSet with 6 elements in 6 sets:
  Set 1 (root: Alice): ['Alice']
  Set 2 (root: Bob): ['Bob']
  Set 3 (root: Carol): ['Carol']
  Set 4 (root: David): ['David']
  Set 5 (root: Eve): ['Eve']
  Set 6 (root: Frank): ['Frank']

Creando amistades...
Alice y Bob son amigos
Bob y Carol son amigos
David y Eve son amigos

Estado después de amistades:
DisjointSet with 6 elements in 3 sets:
  Set 1 (root: Alice): ['Alice', 'Bob', 'Carol']
  Set 2 (root: David): ['David', 'Eve']
  Set 3 (root: Frank): ['Frank']

Consultas:
¿Alice y Carol conectadas? True
¿Alice y David conectadas? False
¿Frank conectado con alguien? False

Uniendo grupos de Alice y David...

Estado final:
DisjointSet with 6 elements in 2 sets:
  Set 1 (root: Alice): ['Alice', 'Bob', 'Carol', 'David', 'Eve']
  Set 2 (root: Frank): ['Frank']

Amigos de Alice: ['Alice', 'Bob', 'Carol', 'David', '

# **Algoritmo de Prim**

In [ ]:
import heapq

def prim(G, v):
  dist = {}
  for v in G:
    dist[v] = float('inf')
  dist[v] = 0
  hq = []
  for v in G:
    heapq.heappush(hq, (dist[v], v))

